In [55]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
from package.databases.initialize import initialize_memories
initialize_memories()

In [33]:
from package.databases.models import User, Project, ProjectDocumentLink, Document, LongTerm, Jargon
from package.databases.session import get_session, Depends
from sqlmodel import Session, select, text
from package.databases.management.user import UserManagement
from package.databases.management.project import ProjectManagement
from package.databases.management.document import DocumentManagement
from package.databases.management.longterm import LongTermManagement
from package.databases.management.jargon import JargonManagement

# CRUD User

In [58]:
user = User(
    username="bank",
    password="555",
    email="bank@bank.com"
)

um = UserManagement()
user = um.create_user(user, session=Depends(get_session))

In [59]:
um.read_user(user.id, session=Depends(get_session))

User(username='bank', id='27b7097c-1c02-425e-922d-0553a4603219', created_at=datetime.datetime(2025, 6, 28, 15, 45, 25, 439086), password='555', email='bank@bank.com', role=<UserRole.USER: 'user'>, updated_at=datetime.datetime(2025, 6, 28, 15, 45, 25, 439086))

In [60]:
user.id

'27b7097c-1c02-425e-922d-0553a4603219'

In [61]:
um.read_user_projects(user.id, session=Depends(get_session))

[]

# CRUD Project

In [62]:
project1 = Project(
    name="Bank Project 1",
    description="A project for banking operations",
    user_id=user.id
)

project2 = Project(
    name="Bank Project 2",
    description="A project for banking operations",
    user_id=user.id
)

pm = ProjectManagement()
project1 = pm.create_project(project1, session=Depends(get_session))
project2 = pm.create_project(project2, session=Depends(get_session))

In [63]:
pm.read_project(project1.id, session=Depends(get_session))

Project(name='Bank Project 1', description='A project for banking operations', created_at=datetime.datetime(2025, 6, 28, 15, 45, 28, 238243), id='27c7f3b3-602d-4e3c-862c-d478716125e3', user_id='27b7097c-1c02-425e-922d-0553a4603219', updated_at=datetime.datetime(2025, 6, 28, 15, 45, 28, 238243))

In [64]:
pm.read_project(project2.id, session=Depends(get_session))

Project(name='Bank Project 2', description='A project for banking operations', created_at=datetime.datetime(2025, 6, 28, 15, 45, 28, 238243), id='2064aa19-ce46-4db5-80b1-ba30411297af', user_id='27b7097c-1c02-425e-922d-0553a4603219', updated_at=datetime.datetime(2025, 6, 28, 15, 45, 28, 238243))

# CRUD Document

In [65]:
document1 = Document(source="Banking Document 1", type="pdf")
document2 = Document(source="Banking Document 2", type="pdf")
document3 = Document(source="Banking Document 3", type="web")
document4 = Document(source="Banking Document 4", type="audio")
document5 = Document(source="Banking Document 5", type="image")

dm = DocumentManagement()
document1 = dm.create_document(document1, session=Depends(get_session)) 
document2 = dm.create_document(document2, session=Depends(get_session))
document3 = dm.create_document(document3, session=Depends(get_session))
document4 = dm.create_document(document4, session=Depends(get_session))
document5 = dm.create_document(document5, session=Depends(get_session))

In [66]:
dm.read_document(document1.id, session=Depends(get_session))

Document(id='90d562cc-6f06-43ea-b79e-c0565edc1d64', status=<DocumentStatus.PENDING: 'pending'>, updated_at=datetime.datetime(2025, 6, 28, 15, 45, 29, 812884), source='Banking Document 1', type='pdf', created_at=datetime.datetime(2025, 6, 28, 15, 45, 29, 812884))

In [67]:
document1.id

'90d562cc-6f06-43ea-b79e-c0565edc1d64'

In [68]:
document1.source

'Banking Document 1'

In [69]:
dm.read_document_longterms(document1.id, session=Depends(get_session))

[]

In [70]:
dm.read_document_jargons(document1.id, session=Depends(get_session))

[]

# CRUD LongTerm

In [71]:
longterm1 = LongTerm(
    raw="Long-term banking strategy 1",
    meta={"source": document1.source},
    document_id=document1.id,
)

longterm2 = LongTerm(
    raw="Long-term banking strategy 2", 
    meta={"source": document1.source},
    document_id=document1.id,
)

longterm3 = LongTerm(
    raw="Long-term banking strategy 3",
    meta={"source": document1.source},
    document_id=document1.id
)

lonterms = [longterm1, longterm2, longterm3]

ltm = LongTermManagement()

ltm.create_raws(lonterms, session=Depends(get_session))

In [72]:
ltm.embed_texts(embed_method="raw", session=Depends(get_session))

In [75]:
ltm.read_similar_text(
    "strategy 2", 
    embed_method="raw", 
    sources=[document1.source, document2.source],
    session=Depends(get_session)
)

[LongTerm(raw_embedding=array([-0.03045654,  0.01348114, -0.04507446, ...,  0.01679993,
         0.00595856, -0.00660324], shape=(1024,), dtype=float32), raw='Long-term banking strategy 2', enrich_embedding=None, combo_embedding=None, document_id='90d562cc-6f06-43ea-b79e-c0565edc1d64', updated_at=datetime.datetime(2025, 6, 28, 15, 45, 39, 944262), enrich=None, id='83f0e655-008c-4de2-bc18-ba43bcd57c0e', combo=None, meta={'source': 'Banking Document 1'}, created_at=datetime.datetime(2025, 6, 28, 15, 45, 38, 893427)),
 LongTerm(raw_embedding=array([-0.02508545, -0.00122166, -0.05023193, ...,  0.02738953,
        -0.00037813, -0.00225067], shape=(1024,), dtype=float32), raw='Long-term banking strategy 3', enrich_embedding=None, combo_embedding=None, document_id='90d562cc-6f06-43ea-b79e-c0565edc1d64', updated_at=datetime.datetime(2025, 6, 28, 15, 45, 39, 944262), enrich=None, id='cbfb9820-727b-4a1a-981a-1a8b1fac7f72', combo=None, meta={'source': 'Banking Document 1'}, created_at=datetime.da

In [50]:
dm.read_document_longterms(document1.id, session=Depends(get_session))

[LongTerm(raw_embedding=array([-0.02508545, -0.00122166, -0.05023193, ...,  0.02738953,
        -0.00037813, -0.00225067], shape=(1024,), dtype=float32), raw='Long-term banking strategy 3', enrich_embedding=None, combo_embedding=None, document_id='9b8c4cb8-0c6b-491f-b441-46c56c5703c6', updated_at=datetime.datetime(2025, 6, 28, 15, 0, 58, 142419), enrich=None, id='081f0eac-c4f0-4690-9405-55b337301f6d', combo=None, meta={'source': 'Banking Document 1'}, created_at=datetime.datetime(2025, 6, 28, 15, 0, 57, 764701)),
 LongTerm(raw_embedding=array([-0.01826477,  0.01348114, -0.06756592, ...,  0.01333618,
         0.00465012, -0.01103973], shape=(1024,), dtype=float32), raw='Long-term banking strategy 1', enrich_embedding=None, combo_embedding=None, document_id='9b8c4cb8-0c6b-491f-b441-46c56c5703c6', updated_at=datetime.datetime(2025, 6, 28, 15, 0, 58, 142419), enrich=None, id='a6bdd28f-ca1a-4b3e-8d7d-3a3f21598076', combo=None, meta={'source': 'Banking Document 1'}, created_at=datetime.datet

# CRUD Jargon

In [51]:
jargon1 = Jargon(
    term="Banking Jargon 1",
    definition="Definition of Banking Jargon 1",
    document_id=document1.id,
    meta={"source": document1.source}
)

jargon2 = Jargon(
    term="Banking Jargon 2",
    definition="Definition of Banking Jargon 2",
    document_id=document1.id,
    meta={"source": document1.source}
)

jargon3 = Jargon(
    term="Banking Jargon 3",
    definition="Definition of Banking Jargon 3",
    document_id=document1.id,
    meta={"source": document1.source}
)

jargons = [jargon1, jargon2, jargon3]

jm = JargonManagement()
jm.create_jargons(jargons, session=Depends(get_session))

In [52]:
dm.read_document_jargons(document1.id, session=Depends(get_session))

[Jargon(term='Banking Jargon 1', definition='Definition of Banking Jargon 1', meta={'source': 'Banking Document 1'}, updated_at=datetime.datetime(2025, 6, 28, 15, 0, 59, 114671), document_id='9b8c4cb8-0c6b-491f-b441-46c56c5703c6', id='d63d08f4-bae1-490e-a408-d901f784af5d', created_at=datetime.datetime(2025, 6, 28, 15, 0, 59, 114671)),
 Jargon(term='Banking Jargon 2', definition='Definition of Banking Jargon 2', meta={'source': 'Banking Document 1'}, updated_at=datetime.datetime(2025, 6, 28, 15, 0, 59, 114671), document_id='9b8c4cb8-0c6b-491f-b441-46c56c5703c6', id='0290f79a-0cbf-4356-b925-476c9022d92d', created_at=datetime.datetime(2025, 6, 28, 15, 0, 59, 114671)),
 Jargon(term='Banking Jargon 3', definition='Definition of Banking Jargon 3', meta={'source': 'Banking Document 1'}, updated_at=datetime.datetime(2025, 6, 28, 15, 0, 59, 114671), document_id='9b8c4cb8-0c6b-491f-b441-46c56c5703c6', id='9afb0735-3cb1-4f55-ba82-cda8dcfad167', created_at=datetime.datetime(2025, 6, 28, 15, 0, 59,

# Clean up

In [76]:
from package.databases.destroy import drop_all_tables

drop_all_tables()

✅ All tables dropped.
